In [1]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import numpy_indexed as npi
from scipy.sparse import lil_matrix
from sklearn.preprocessing import normalize

In [2]:
Path="/home/mgander/mouse_atlas/data/WOT"
days=list(pd.read_csv(f'{Path}/Representations/obs/days.csv')['0'])

# Load the neirest neighbors and restructure them into a sparse matrix

By running TOME we get csv-files with the neirest neighbors. This accumulates all these nn and shapes it into sparse matrices, that are then considered the analogous to moscots transport matrices.

In [3]:
def cellID_Dict(obs):
    cellids=list(obs['Unnamed: 0'])
    
    D={cellids[i]:i for i in range(len(cellids))}
    return(D)

def construct_nn_matrix(ts0, D0, D1, j):
    
    n=pd.read_csv(f'{Path}/TOME//{ts0}/neigbors_{j}.csv')
    c0=list(pd.read_csv(f'{Path}/TOME/{ts0}/early_cells_{j}.csv')['x'])
    c1=list(pd.read_csv(f'{Path}/TOME/{ts0}/late_cells_{j}.csv')['x'])

    D0s={}
    for i in range(len(c0)):
        D0s[i+1]=D0[c0[i]]
    D1s={}
    for i in range(len(c1)):
        D1s[i]=D1[c1[i]]

    N=n.values.copy()
    ar=N.flatten().copy()
    Nn = npi.remap(ar, list(D0s.keys()), list(D0s.values()))
    Nn=Nn.reshape(N.shape)
    return(Nn, D1s)

In [4]:
def TOME_output_to_transport_matrix(obs0, obs1, ts0):
    D0=cellID_Dict(obs0)
    D1=cellID_Dict(obs1)
    
    M = lil_matrix((len(obs0), len(obs1)), dtype=int)

    for j in range(1,501):
        if j%100==0:
            print(j)
        Nn, D1s=construct_nn_matrix(ts0, D0, D1, j)

        for k in range(len(Nn)):
            for ks in range(Nn.shape[1]):
                M[Nn[k,ks], D1s[k]]+=1
    return(M)

In [6]:
for i in range(len(days)-1):
    ts0=days[i]
    ts1=days[i+1]
    print(ts0)

    obs0=pd.read_csv(f"{Path}/Representations/obs/{ts0}.csv")
    obs1=pd.read_csv(f"{Path}/Representations/obs/{ts1}.csv")

    M=TOME_output_to_transport_matrix(obs0, obs1, ts0)
    #scipy.sparse.save_npz(f'{Path}/TOME_maps_growth_rate_and_driver_genes_analysis/{ts0}.npz', M.tocsr())


    M = normalize(M, norm='l1', axis=0)
    gr=np.asarray(M.sum(1).T)[0]

    df=pd.DataFrame({'CellID':list(obs0['Unnamed: 0']), 'TOME_growth_rates':gr})
    df.index=list(obs0['Unnamed: 0'])
    df.to_pickle(f'{Path}/growth_rates/TOME/{ts0}.pkl')

8.0
100
200
300
400
500
8.25
100
200
300
400
500
8.5
100
200
300
400
500
8.75
100
200
300
400
500
9.0
100
200
300
400
500
9.5
100
200
300
400
500
10.0
100
200
300
400
500
10.5
100
200
300
400
500
11.0
100
200
300
400
500
11.5
100
200
300
400
500
12.0
100
200
300
400
500
12.5
100
200
300
400
500
13.0
100
200
300
400
500
13.5
100
200
300
400
500
14.0
100
200
300
400
500
14.5
100
200
300
400
500
15.0
100
200
300
400
500
15.5
100
200
300
400
500
16.0
100
200
300
400
500
16.5
100
200
300
400
500
17.0
100
200
300
400
500
17.5
100
200
300
400
500
